<a href="https://colab.research.google.com/github/avillemin/One-Shot-Learning-for-Face-Recognition/blob/master/One_Shot_Learning_for_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class VGG_16(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_size = [2, 2, 3, 3, 3]
        self.conv1_1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv5_1 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.fc6 = nn.Linear(512 * 7 * 7, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 2622)

    def forward(self, x):
        """ Pytorch forward
        Args:
            x: input image (224x224)
        Returns: class logits
        """
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, self.training)
        x = self.fc7(x)
        # x = F.dropout(x, 0.5, self.training)
        # x = self.fc8(x)
        return x


In [26]:
from PIL import Image

class Network():
    def __init__(self, load_path = None):
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        print("Classifier running on GPU" if self.cuda else "Classifier running on CPU")
        self.model = VGG_16().to(self.device)
        if load_path!=None:
            self.model.load_state_dict(torch.load(load_path))
        self.loss_f = nn.L1Loss()

    def predict(self, im1, im2):
        self.model.eval()
        with torch.no_grad():
            output1 = self.model(self.preprocess_image(im1).unsqueeze(0))
            output2 = self.model(self.preprocess_image(im2).unsqueeze(0))
        return self.loss_f(output1,output2)

    def preprocess_image(self, img):
        img = img.resize((224, 224), Image.ANTIALIAS)
        try:
            img = np.array([img.getdata(0),img.getdata(1),img.getdata(2)])
        except:
            img = np.array([img.getdata(0),img.getdata(0),img.getdata(0)])
        
        img = torch.tensor(img.reshape(3,224,224)).float().to(self.device)
        return img

classifier = Network('/content/gdrive/My Drive/CodeML/vgg_face_dag.pth')

Classifier running on GPU


In [0]:
from PIL import Image
im1 = Image.open('/content/gdrive/My Drive/CodeML/predict/74fdcc320a503148.jpg')

In [0]:
import glob
result = []
for pred_file in glob.glob("/content/gdrive/My Drive/CodeML/predict/*"):
    pred_img = Image.open(pred_file)
    mini = 1000 
    predict = None
    for file in glob.glob("/content/gdrive/My Drive/CodeML/identity/*"):
        try:
            im2 = Image.open(file)
            score = classifier.predict(im1,im2)
            if score<mini:
                mini = score
                predict = file
        except:
            pass
    result.append([pred_file.split('/')[-1],predict])